# GO QuickGO API access
---

Natalia García Sánchez
16/11/2023

Acquisition / retrieval of GO terms relevant to sulfate metabolism - epigenetics - autophagy

In [1]:
import requests
import pandas as pd
import io

In [2]:
# Some code to return a Pandas dataframe, given tabular text
def to_df(result):
    return pd.read_table(io.StringIO(result), header=None)

In [3]:
request_url = "http://pantherdb.org/services/oai/pantherdb/supportedgenomes"
res = requests.post(request_url, {'caller_identity':'https://github.com/Natpod/Bnapus_ME'})

version=res.json()['search']['product']

In [4]:
base_url = "https://www.ebi.ac.uk/QuickGO/services"
etc = "/ontology/go/about"
request_url = base_url+etc
r = requests.get(request_url, headers={ "Accept" : "application/json"})

if not r.ok:
  r.raise_for_status()
  sys.exit()

responseBody = r.text
print(responseBody)

{"go":{"version":"http://purl.obolibrary.org/obo/go/releases/2023-11-06/extensions/go-plus.owl","timestamp":"2023-11-06"}}


In [5]:
import requests, sys
import json

requestURL = "https://www.ebi.ac.uk/QuickGO/services/ontology/go/search?query=apopto||sulfur&limit=25&page=1"

r = requests.get(requestURL, headers={ "Accept" : "application/json"})

if not r.ok:
  r.raise_for_status()
  sys.exit()

responseBody = r.json()

In [6]:
import requests, sys
import json

requestURL = "https://www.ebi.ac.uk/QuickGO/services/ontology/go/search?query=sulfur||met&page=1"

r = requests.get(requestURL, headers={ "Accept" : "application/json"})

if not r.ok:
  r.raise_for_status()
  sys.exit()

responseBody = r.json()
res_ap = pd.DataFrame(responseBody["results"])
colnames = res_ap.columns[:].tolist()
res_all = pd.DataFrame(columns=colnames)


for page in range(1, (responseBody["numberOfHits"] // 25)+2):
    requestURL = f"https://www.ebi.ac.uk/QuickGO/services/ontology/go/search?query=sulfur||met&page={page}"

    r = requests.get(requestURL, headers={ "Accept" : "application/json"})

    if not r.ok:
      r.raise_for_status()
      sys.exit()

    responseBody = r.json()
    res_all = pd.concat((res_all, pd.DataFrame(responseBody["results"])), ignore_index=True)

In [7]:
res_all.groupby("aspect").describe()["id"]

,count,unique,top,freq
aspect,,,,
biological_process,112,16,GO:0000098,7
molecular_function,63,9,GO:0033796,7


In [8]:
bp_terms = res_all[res_all["aspect"] == "biological_process"]
mf_terms = res_all[res_all["aspect"] == "molecular_function"]

In [9]:
bp_terms

,id,isObsolete,name,definition,aspect
0,GO:0006791,False,sulfur utilization,{'text': 'A series of processes that forms an ...,biological_process
1,GO:0019417,False,sulfur oxidation,{'text': 'The chemical reactions and pathways ...,biological_process
2,GO:0070220,False,aerobic sulfur oxidation,{'text': 'A sulfur oxidation process that proc...,biological_process
5,GO:0072348,False,sulfur compound transport,{'text': 'The directed movement of compounds t...,biological_process
8,GO:1902497,False,iron-sulfur cluster transmembrane transport,{'text': 'A process in which an iron-sulfur cl...,biological_process
...,...,...,...,...,...
170,GO:0044272,False,sulfur compound biosynthetic process,{'text': 'The chemical reactions and pathways ...,biological_process
171,GO:0044273,False,sulfur compound catabolic process,{'text': 'The chemical reactions and pathways ...,biological_process
172,GO:0019422,False,disproportionation of elemental sulfur,{'text': 'The process in which sulfur compound...,biological_process
173,GO:0070223,False,"sulfide oxidation, using sulfur dioxygenase",{'text': 'A sulfide oxidation process that pro...,biological_process


#https://www.ebi.ac.uk/QuickGO/api/index.html#!/gene_ontology/findChildrenUsingGET_1
#https://www.ebi.ac.uk/QuickGO/api/index.html#!/gene_ontology/getChartUsingGET_1

GET /ontology/go/terms/{ids}/chart
GET /ontology/go/terms/{ids}/children
GET /ontology/go/terms/{ids}/descendants
GET /ontology/go/terms/{ids}/constraints


In [10]:
mf_terms

,id,isObsolete,name,definition,aspect
3,GO:1901681,False,sulfur compound binding,{'text': 'Binding to a sulfur compound.'},molecular_function
4,GO:0033796,False,sulfur reductase activity,{'text': 'Catalysis of the reduction of elemen...,molecular_function
6,GO:0050313,False,sulfur dioxygenase activity,{'text': 'Catalysis of the reaction: S-sulfany...,molecular_function
7,GO:0097163,False,sulfur carrier activity,{'text': 'Covalently binding to sulfur and del...,molecular_function
11,GO:0051536,False,iron-sulfur cluster binding,"{'text': 'Binding to an iron-sulfur cluster, a...",molecular_function
...,...,...,...,...,...
161,GO:0051536,False,iron-sulfur cluster binding,"{'text': 'Binding to an iron-sulfur cluster, a...",molecular_function
162,GO:0016846,False,carbon-sulfur lyase activity,{'text': 'Catalysis of the elimination of hydr...,molecular_function
165,GO:0043826,False,sulfur oxygenase reductase activity,{'text': 'Catalysis of the reaction: 5 sulfur ...,molecular_function
166,GO:0033755,False,sulfur oxygenase/reductase activity,{'text': 'Catalysis of the reaction: 4 sulfur ...,molecular_function


In [11]:
for GO in bp_terms["id"].values.tolist():

    requestURL = f"https://www.ebi.ac.uk/QuickGO/services/annotation/search?includeFields=goName&selectedFields=geneProductId&selectedFields=symbol&selectedFields=evidenceCode&taxonId=3708&goId={GO}"


    r = requests.get(requestURL, headers={ "Accept" : "application/json"})

    if not r.ok:
      r.raise_for_status()
      sys.exit()

    responseBody = r.json()
    df_go_specific_genes = pd.DataFrame(responseBody["results"])


    for page in range(1, (responseBody["numberOfHits"] // 25)+2):
        requestURL = f"https://www.ebi.ac.uk/QuickGO/services/annotation/search?includeFields=goName&selectedFields=geneProductId&selectedFields=symbol&selectedFields=evidenceCode&taxonId=3708&goId=GO:0006790&page={page}"
        r = requests.get(requestURL, headers={ "Accept" : "application/json"})
        if not r.ok:
          r.raise_for_status()
          sys.exit()

        responseBody = r.json()
        res_all = pd.concat((res_all, pd.DataFrame(responseBody["results"])), ignore_index=True)



In [12]:
res_all

,id,isObsolete,name,definition,aspect,geneProductId,qualifier,goId,goName,goEvidence,...,reference,withFrom,taxonId,taxonName,assignedBy,extensions,targetSets,symbol,date,synonyms
0,GO:0006791,False,sulfur utilization,{'text': 'A series of processes that forms an ...,biological_process,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,GO:0019417,False,sulfur oxidation,{'text': 'The chemical reactions and pathways ...,biological_process,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,GO:0070220,False,aerobic sulfur oxidation,{'text': 'A sulfur oxidation process that proc...,biological_process,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,GO:1901681,False,sulfur compound binding,{'text': 'Binding to a sulfur compound.'},molecular_function,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,GO:0033796,False,sulfur reductase activity,{'text': 'Catalysis of the reduction of elemen...,molecular_function,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21940,UniProtKB:A0A078F7N9!131517490,NaN,None,NaN,NaN,UniProtKB:A0A078F7N9,involved_in,GO:0019344,cysteine biosynthetic process,IEA,...,GO_REF:0000043,"[{'connectedXrefs': [{'db': 'UniProtKB-KW', 'i...",3708.0,None,UniProt,None,None,BnaC08g09930D,20230911,None
21941,UniProtKB:A0A078F7N9!131518892,NaN,None,NaN,NaN,UniProtKB:A0A078F7N9,enables,GO:0004124,cysteine synthase activity,IEA,...,GO_REF:0000003,"[{'connectedXrefs': [{'db': 'EC', 'id': '2.5.1...",3708.0,None,UniProt,None,None,BnaC08g09930D,20230911,None
21942,UniProtKB:A0A078F7N9!131521218,NaN,None,NaN,NaN,UniProtKB:A0A078F7N9,enables,GO:0004124,cysteine synthase activity,IEA,...,GO_REF:0000116,"[{'connectedXrefs': [{'db': 'RHEA', 'id': '148...",3708.0,None,RHEA,None,None,BnaC08g09930D,20230911,None
21943,UniProtKB:A0A078F7N9!131524502,NaN,None,NaN,NaN,UniProtKB:A0A078F7N9,enables,GO:0004124,cysteine synthase activity,IEA,...,GO_REF:0000104,"[{'connectedXrefs': [{'db': 'UniRule', 'id': '...",3708.0,None,UniProt,None,None,BnaC08g09930D,20230911,None
